## Favorite Coffee Predictive Model

Goal: Build a model that predicts which of four coffee samples a person would like. Identify features that could be included in a survey

### Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn import tree
import scipy.stats as st
from sklearn.preprocessing import OneHotEncoder

### Data Pull

In [3]:
coffee_df = pd.read_csv("GACTT_RESULTS_ANONYMIZED.csv")
coffee_df.head()

,Submission ID,Respondent ID,Submitted at,What is your age?,What is your ZIP code?,How many cups of coffee do you typically drink per day?,Where do you typically drink coffee?,Where do you typically drink coffee? (At home),Where do you typically drink coffee? (At the office),Where do you typically drink coffee? (On the go),...,Gender,Gender (please specify),Marital Status,Household Income,Education Level,Ethnicity/Race,Ethnicity/Race (please specify),Employment Status,Number of Children,Political Affiliation
0,gMR29l,vDzkBA,9/11/23 20:50,18-24 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BkPN0e,9Z4VME,9/11/23 20:52,25-34 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,W5G8jj,XoNLQj,9/12/23 1:33,25-34 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4xWgGr,Gp7gde,9/12/23 12:58,35-44 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,QD27Q8,Bp9ogQ,9/12/23 15:24,25-34 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Cleaning

In [ ]:
# select columns that may be good predictors
good_cols = {
    "What is your age?":  'age',
    "How many cups of coffee do you typically drink per day?": 'typical_cups_per_day',
    "Where do you typically drink coffee?": 'typical_drink_location',
    "What is your favorite coffee drink?": 'favorite_drink',
    "Do you usually add anything to your coffee?": 'usual_additions',
    "Before today's tasting, which of the following best described what kind of coffee you like?": 'liked_coffee_kind',
    "How strong do you like your coffee?": 'preffered_strength',
    "What roast level of coffee do you prefer?": 'preffered_roast',
    "How much caffeine do you like in your coffee?": 'preffered_caffeine',
    "In total, much money do you typically spend on coffee in a month?": 'monthly_expense',
    "Why do you drink coffee?": 'why_drink',
    "Do you like the taste of coffee?": 'like_taste',
    "Do you know where your coffee comes from?": 'know_orgin',
    "What is the most you've ever paid for a cup of coffee?": 'most_paid',
    "What is the most you'd ever be willing to pay for a cup of coffee?": 'most_willing_pay',
    "Do you feel like you’re getting good value for your money when you buy coffee at a cafe?": 'cafe_feel_good_value',
    "Approximately how much have you spent on coffee equipment in the past 5 years?": 'equipment_expense_5_years',
    "Do you feel like you’re getting good value for your money with regards to your coffee equipment?": 'equipment_feel_good_value',
    
    "Lastly, what was your favorite overall coffee?": 'taste_test_favorite'
}

# get subset of useful columns
coffee_df = coffee_df[good_cols.keys()]

# rename useful columns
coffee_df.columns = good_cols.values()

# remove surveys with missing favorite coffee
coffee_df = coffee_df.query('taste_test_favorite.notna()')

# reset index
coffee_df.reset_index(inplace = True,drop = True)

coffee_df.head()

### Data Analysis

In [ ]:
## Plot distribution of favorite coffees
coffee_df.taste_test_favorite.value_counts()[['Coffee A','Coffee B','Coffee C','Coffee D']].plot(kind='bar')
plt.xticks(rotation=0)
plt.xlabel('Coffee')
plt.ylabel('Frequency')
plt.title('Distribution of Taste Test Favorite')

In [ ]:
## Plot proportion of missing values
pd.isnull(coffee_df).mean().sort_values(ascending = False).plot(kind = 'barh',
                                                                title = 'Proportion of Missing Values by Feature',
                                                                xlabel = 'Proportion Missing',
                                                                ylabel = 'Feature'
                                                               )
plt.xlim(0,1)

In [ ]:
## Perform chi squared test
chisquare_p_vals = {}

for col in coffee_df.drop('taste_test_favorite',axis=1).columns:
    chisquare_p_vals[col] = st.chi2_contingency(pd.crosstab(coffee_df[col], coffee_df.taste_test_favorite, dropna = False)).pvalue
    
chisquare_p_vals = pd.DataFrame(chisquare_p_vals.values(), index = chisquare_p_vals.keys())

chisquare_p_vals.sort_values(by=0,ascending=False).plot(kind = 'barh',
                                       title = 'Chi Squared Tests P Values',
                                       ylabel = 'Feature',
                                       xlabel = 'P Value',
                                       legend = False)

## Encode Categorical Features

In [ ]:
## specify ordinal, multi select,  and categorical columns
ordinal_cols = ['age','typical_cups_per_day','preffered_strength','preffered_caffeine','monthly_expense','most_paid','most_willing_pay','equipment_expense_5_years']
mul_select_cols = ['typical_drink_location','usual_additions','why_drink']
cat_cols = coffee_df.drop(ordinal_cols, axis = 1).drop(mul_select_cols,axis=1).drop('taste_test_favorite',axis=1).columns

In [ ]:
## map ordinal columns to 0-n
ordinal_cols_map = {
    'age': ['<18 years old','18-24 years old','25-34 years old','35-44 years old','45-54 years old','55-64 years old','>65 years old'],
    'typical_cups_per_day': [np.nan, 'Less than 1', '1', '2','3','4','More than 4'],
    'preffered_strength': [np.nan, 'Weak','Somewhat light','Medium','Somewhat strong','Very strong'],
    'preffered_caffeine': [np.nan,'Decaf','Half caff','Full caffeine'],
    'monthly_expense': [np.nan, '<$20','$20-$40','$40-$60','$60-$80','$80-$100','>$100'],
    'most_paid': [np.nan, 'Less than $2','$2-$4','$4-$6','$6-$8','$8-$10','$10-$15','$15-$20','More than $20'],
    'most_willing_pay': [np.nan, 'Less than $2','$2-$4','$4-$6','$6-$8','$8-$10','$10-$15','$15-$20','More than $20'],
    'equipment_expense_5_years': [np.nan, 'Less than $20','$20-$50','$50-$100','$100-$300','$300-$500','$500-$1000','More than $1,000']   
}

ordinal_coffee_df = coffee_df[ordinal_cols]

for ordinal_col in ordinal_cols_map.keys():
    ordinal_map = dict(zip(ordinal_cols_map[ordinal_col], np.arange(len(ordinal_cols_map[ordinal_col]))))
    ordinal_coffee_df[ordinal_col] = ordinal_coffee_df[ordinal_col].map(ordinal_map)

ordinal_coffee_df.head()

In [ ]:
## one hot encode special multiple selection columns
mul_select_cols = {'typical_drink_location': ['None of these','At home','At a cafe','On the go','At the ofice'],
                   'usual_additions': ['No - just black','Milk / dairy alternative','coffee creamer','Sugar / sweetener','Flavor syrup','Other'],
                   'why_drink': ['It tastes good','I need the caffeine','I need the ritual',' It makes me go to the bathroom','Other']}

mul_select_ohe_df = pd.DataFrame()

for col in mul_select_cols.keys():
    for val in mul_select_cols[col]:
        mul_select_ohe_df[col + '_' + val] = coffee_df[col].apply(lambda x: 0 if pd.isna(x) else 1 if val in x else 0) 

In [ ]:
## one hot encode categorical features
ohe = OneHotEncoder()
ohe_results = ohe.fit_transform(coffee_df[cat_cols])
coffee_ohe_df = pd.DataFrame(ohe_results.toarray(), columns = ohe.get_feature_names_out())
coffee_ohe_df.head()

In [ ]:
## merge ordinal, one hot encoded, multiple selection one hot encoded, and target feature 
coffee_df = pd.concat([ordinal_coffee_df, coffee_ohe_df, mul_select_ohe_df, coffee_df.taste_test_favorite],axis=1)
coffee_df.head()

## Train/Test Split

In [ ]:
## 80/20 train test split
train_df = coffee_df.sample(frac = 0.8)
test_df = coffee_df.drop(train_df.index)

train_x = train_df.drop('taste_test_favorite',axis=1)
train_y = train_df.taste_test_favorite
test_x = test_df.drop('taste_test_favorite',axis=1)
test_y = test_df.taste_test_favorite

print(f"Train Shape: {train_df.shape}")
print(f"Test Shape: {test_df.shape}")

## Baseline

In [ ]:
## get baseline model accuracy
(coffee_df.taste_test_favorite.value_counts()/coffee_df.shape[0])[['Coffee A','Coffee B', 'Coffee C','Coffee D']].plot(kind = 'bar',
                                                                                                                       xlabel = 'Taste Test Favorite',
                                                                                                                       ylabel = 'Proportion',
                                                                                                                       title = 'Taste Test Favorite Proportions')
plt.xticks(rotation=0)

## Initial Model

In [ ]:
## train initial model
clf = tree.DecisionTreeClassifier(max_depth = 4)
clf.fit(train_x,train_y)

train_acc = np.mean(train_y == clf.predict(train_x))
test_acc = np.mean(test_y == clf.predict(test_x))

print(f"Train Accuracy: {int(100*train_acc)}%")
print(f"Test Accuracy: {int(100*test_acc)}%")

## Hyperparameter Tuning

In [ ]:
## hyperparameter tune max_depth 
hyperparameter_tuning_df = pd.DataFrame({
                                         'depth': [],
                                         'train_accuracy': [],
                                         'test_accuracy': []
                                        })

md = 20
depths = np.arange(1,md)
for depth in depths:
    clf = tree.DecisionTreeClassifier(max_depth = depth)
    clf.fit(train_x,train_y)
    
    train_acc = np.mean(train_y == clf.predict(train_x))
    test_acc = np.mean(test_y == clf.predict(test_x))
    
    hyperparameter_tuning_df.loc[len(hyperparameter_tuning_df.index)] = [depth, train_acc, test_acc]

plt.plot(hyperparameter_tuning_df.depth, hyperparameter_tuning_df.train_accuracy, label = 'Train Accuracy')
plt.plot(hyperparameter_tuning_df.depth, hyperparameter_tuning_df.test_accuracy, label = 'Test Accuracy')
plt.plot([1,md],[0.37,0.37],label='Baseline',ls='--')
plt.xticks(np.arange(1,md))
plt.legend()
plt.yticks(np.arange(0,1.1,0.1))
plt.xlabel('Tree Max Depth')
plt.ylabel('Accuracy')
plt.title('Accuacy by Max Depth')
plt.ylim(0,1)

## Final Model Performance

In [ ]:
# 10 Fold Cross Validation
cvs=10

train_accs = []
test_accs = []

coffee_df = coffee_df.sample(frac=1)

for i in range(cvs):
    
    test_df = coffee_df[i*377:(i+1)*377]
    train_df = coffee_df.drop(test_df.index)
    
    train_x = train_df.drop('taste_test_favorite',axis=1)
    train_y = train_df.taste_test_favorite
    test_x = test_df.drop('taste_test_favorite',axis=1)
    test_y = test_df.taste_test_favorite
    
    clf = tree.DecisionTreeClassifier(max_depth = 2)
    clf.fit(train_x,train_y)
    
    train_acc = np.mean(train_y == clf.predict(train_x))
    test_acc = np.mean(test_y == clf.predict(test_x))
    
    train_accs = np.append(train_accs, train_acc)
    test_accs = np.append(test_accs, test_acc)

plt.scatter(train_accs,[1]*10,label = 'Train Accuracy')
plt.scatter(test_accs,[1]*10, label = 'Test Accuracy')
plt.title('10 Fold Cross Validation Accuracies')
plt.yticks([1],[''])
plt.xlabel('Accuracy')
plt.legend()
plt.xlim(0.4,0.45)

In [ ]:
np.mean(test_accs)

In [ ]:
## feature importance
feature_importance_df = pd.DataFrame({'col':train_x.columns,
                                      'feature_importance':clf.feature_importances_
                                     })
feature_importance_df.sort_values('feature_importance',ascending=False).head(10)

In [ ]:
## tree plot
plt.figure(figsize=(50,30))
sk.tree.plot_tree(clf,
                  feature_names = train_x.columns,
                  class_names = clf.classes_,
                  filled = True,
                  impurity = False,
                  proportion = True,
                  precision = 2
                 )
plt.savefig('final_model_tree.png')